# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [6]:
%idle_timeout 15
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2
# %additional_python_modules PyAthena,pyarrow==7,awswrangler
# %extra_jars [s3://pb-4-0-datalake-transformed-layer/rfm_demo/iceberg-spark-runtime-3.3_2.12-1.4.3.jar,s3://pb-4-0-datalake-transformed-layer/rfm_demo/iceberg-aws-bundle-1.4.3.jar]
%%configure
{
    "--conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.delta.logStore.class=org.apache.spark.sql.delta.storage.S3SingleDriverLogStore",
    "--datalake-formats": "delta"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.2 
Current idle_timeout is None minutes.
idle_timeout has been set to 15 minutes.
Setting Glue version to: 4.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
The following configurations have been updated: {'--conf': 'spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.delta.logStore.class=org.apache.spark.sql.delta.storage.S3SingleDriverLogStore', '--datalake-formats': 'delta'}


####  Run this cell to set up and start your interactive session.


In [1]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, DoubleType, BooleanType, ArrayType, DateType
from pyspark.sql.types import StructType, StructField, ArrayType, StringType, IntegerType, LongType, DoubleType, BooleanType
from pyspark.sql.functions import col, count, when, sum, min, max, expr
from pyspark.sql import functions as F
import boto3

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Session ID: ae876d3a-4708-4efb-964a-677e8ab57c36
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog --conf spark.delta.logStore.class=org.apache.spark.sql.delta.storage.S3SingleDriverLogStore
--datalake-formats delta
Waiting for session ae876d3a-4708-4efb-964a-677e8ab57c36 to get into ready status...
Session ae876d3a-4708-4efb-964a-677e8ab57c36 has been created.



In [11]:
# source 
s3_path = "s3://robin-poc-bucket/rahul/test_1/hands_json_data/"
# s3 location for database 
delta_database = "pokershot_sample_delta_database_v1"
hands_gold_data_s3_location = f"s3://robin-poc-bucket/rahul/{delta_database}/"

# tables and s3 location and check_point_path
bronze_table = "hands_bronze_data"
bronze_table_s3_location = f"{hands_gold_data_s3_location}{bronze_table}/"
bronze_checkpointlocation = "s3://robin-poc-bucket/rahul/test_1/bronze_checkpointlocation/"

gold_table = "hands_gold_data"
gold_table_s3_location = f"{hands_gold_data_s3_location}{gold_table}/"
gold_checkpointlocation = "s3://robin-poc-bucket/rahul/test_1/gold_checkpointlocation/"

# explicity checkpoint location
external_checkpoint_table = "external_checkpoints"
external_checkpoint_table_s3_location = f"{hands_gold_data_s3_location}{external_checkpoint_table}/"
explicit_checkpoint_path = "s3://robin-poc-bucket/rahul/test_1/external_checkpointlocation/"

# open search index
opensearch_index = "performance_by_stake_v2"
opensearch_nodes = "https://vpc-testpspvt-2xjjsrwejwnaxv7l3zoavngysy.ap-south-1.es.amazonaws.com"
opensearch_checkpointlocation = "s3://robin-poc-bucket/rahul/test_1/opensearch_checkpointlocation/"

#TODO startingTimestamp in gold layer 

# create database
spark.sql(f"Create database if not exists {delta_database} location '{hands_gold_data_s3_location}' ")

DataFrame[]


#### Example: Create a DynamicFrame from a table in the AWS Glue Data Catalog and display its schema


In [3]:
spark.sql(f"Create database if not exists {delta_database} location '{hands_gold_data_s3_location}' ").show()

++
||
++
++


#### Example: Convert the DynamicFrame to a Spark DataFrame and display a sample of the data


In [12]:
class Bronze():
    
    def __init__(self):
        self.s3_path = s3_path
        self.delta_database = delta_database
        self.bronze_table = bronze_table
        self.bronze_table_s3_location = bronze_table_s3_location
        self.bronze_checkpointlocation = bronze_checkpointlocation

    def get_schema(self):
        SCHEMA = StructType([
            StructField("event_data", StructType([
                StructField("community_cards", ArrayType(StructType([
                    StructField("card_suit", StringType()),
                    StructField("card_value", IntegerType())
                ]))),
                StructField("community_cards_rit", ArrayType(StructType([
                    StructField("card_suit", StringType()),
                    StructField("card_value", IntegerType())
                ]))),
                StructField("game_end_timestamp", IntegerType()),
                StructField("game_start_timestamp", IntegerType()),
                StructField("hand_id", LongType()),
                StructField("player_details", ArrayType(StructType([
                    StructField("cards", ArrayType(StructType([
                        StructField("card_suit", StringType()),
                        StructField("card_value", IntegerType())
                    ]))),
                    StructField("chip_stack_end", DoubleType()),
                    StructField("chip_stack_start", DoubleType()),
                    StructField("position", StringType()),
                    StructField("profit_loss", DoubleType()),
                    StructField("reference_number", StringType()),
                    StructField("seat_index", IntegerType()),
                    StructField("session_id", StringType()),
                    StructField("total_bets", DoubleType()),
                    StructField("total_win", DoubleType()),
                    StructField("user_id", LongType()),
                    StructField("username", StringType()),
                    StructField("rake", StructType([
                        StructField("bonus_rake", DoubleType()),
                        StructField("net_rake", DoubleType()),
                        StructField("service_tax", DoubleType()),
                        StructField("total_rake", DoubleType())
                    ]))
                ]))),
                StructField("players_in_hand", IntegerType()),
                StructField("rit_game", IntegerType()),
                StructField("round_details", StructType([
                    StructField("flop", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ])),
                    StructField("preflop", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ])),
                    StructField("river", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ])),
                    StructField("showdown", StructType([
                        StructField("pot_details", ArrayType(DoubleType())),
                        StructField("pot_details_rit", ArrayType(DoubleType())),
                        StructField("pot_winners", ArrayType(StructType([
                            StructField("pot_amount", DoubleType()),
                            StructField("pot_contestants", ArrayType(IntegerType())),
                            StructField("winner_details", ArrayType(StructType([
                                StructField("amount", DoubleType()),
                                StructField("timestamp", IntegerType()),
                                StructField("user_id", IntegerType()),
                                StructField("win_type", StringType())
                            ])))
                        ]))),
                        StructField("pot_winners_rit", ArrayType(StructType([
                            StructField("pot_amount", DoubleType()),
                            StructField("pot_contestants", ArrayType(IntegerType())),
                            StructField("winner_details", ArrayType(StructType([
                                StructField("amount", DoubleType()),
                                StructField("timestamp", IntegerType()),
                                StructField("user_id", IntegerType()),
                                StructField("win_type", StringType())
                            ])))
                        ])))
                    ])),
                    StructField("turn", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ]))
                ])),
                StructField("showdown", BooleanType()),
                StructField("table_info", StructType([
                    StructField("big_blind", DoubleType()),
                    StructField("is_anonymous", BooleanType()),
                    StructField("max_buyin", DoubleType()),
                    StructField("min_buyin", DoubleType()),
                    StructField("mini_game_type_id", StringType()),
                    StructField("players_per_table", IntegerType()),
                    StructField("small_blind", DoubleType()),
                    StructField("table_child_id", IntegerType()),
                    StructField("table_id", IntegerType()),
                    StructField("table_type", StringType())
                ])),
                StructField("total_pot", DoubleType()),
                StructField("user_stats", ArrayType(StructType([
                    StructField("3bet", BooleanType()),
                    StructField("3bet_opportunity", BooleanType()),
                    StructField("all_in", BooleanType()),
                    StructField("allin_opportunity", BooleanType()),
                    StructField("flop_seen", BooleanType()),
                    StructField("fold_opportunity", BooleanType()),
                    StructField("hp", BooleanType()),
                    StructField("pfr_count", BooleanType()),
                    StructField("pfr_opportunity", BooleanType()),
                    StructField("pr", BooleanType()),
                    StructField("time_played", IntegerType()),
                    StructField("user_id", IntegerType()),
                    StructField("user_won", BooleanType()),
                    StructField("vpip", BooleanType()),
                    StructField("vpip_opportunity", BooleanType()),
                    StructField("won_at_showdown", BooleanType()),
                    StructField("wtsd", BooleanType()),
                    StructField("fold_on_allin_opportunity", BooleanType()),
                    StructField("fold", BooleanType()),
                    StructField("3bet_fold_opportunity", BooleanType()),
                    StructField("steal_opportunity", BooleanType()),
                    StructField("steal_attempt", BooleanType()),
                    StructField("fold_on_allin", BooleanType()),
                    StructField("check_n_raise_opportunity", BooleanType()),
                    StructField("cbet_opportunity", BooleanType()),
                    StructField("cbet_success", BooleanType()),
                    StructField("fold_to_cbet_opportunity", BooleanType()),
                    StructField("folded_to_cbet", BooleanType()),
                    StructField("folded_to_3bet", BooleanType()),
                    StructField("check_n_raise", BooleanType())
                ]))),
                StructField("game_rake", StructType([
                    StructField("bonus_rake", DoubleType()),
                    StructField("net_rake", DoubleType()),
                    StructField("service_tax", DoubleType()),
                    StructField("total_rake", DoubleType())
                ]))
            ])),
            StructField("event_name", StringType()),
            StructField("event_source", StringType()),
            StructField("event_timestamp", IntegerType()),
            StructField("event_uuid", StringType()),
            StructField("event_version", StringType())
        ])
        
        return SCHEMA


    def readHandsEvents(self):
        schema = self.get_schema()
        return spark.readStream.format("json").schema(schema).load(self.s3_path)
                
    def writeToDeltaTable(self, hands_df):
        
        additional_options = {
            "path": self.bronze_table_s3_location
        }
        
        sbronzeQuery = ( 
            hands_df.writeStream
                    .queryName("bronze-ingestion")
                    .format("delta")
                    .options(**additional_options)
                    .option("checkpointLocation", self.bronze_checkpointlocation)
                    .outputMode("append")
                    .option("mergeSchema", "true")
                    .partitionBy("event_name", "record_date")
                    .toTable(f"{self.delta_database}.{self.bronze_table}")
        ) 
        return sbronzeQuery

    def get_file_metadata(self, raw_data_df):
        raw_data_df = raw_data_df.select("*", "_metadata.file_modification_time", "_metadata.file_path").withColumn("record_date", expr("date(cast(event_timestamp as timestamp))"))
        return raw_data_df

    def process(self):
        raw_data_df = self.readHandsEvents()
        raw_data_with_file_meta_df = self.get_file_metadata(raw_data_df)
        sQuery = self.writeToDeltaTable(raw_data_with_file_meta_df)
        return raw_data_with_file_meta_df

#### Example: Visualize data with matplotlib


In [13]:
print("Started Bronze processing ")
br = Bronze()
raw_data_df = br.process()
# b.awaitTermination()
print("Stopped bronze processing ")

Started Bronze processing 
Stopped bronze processing


#### Example: Write the data in the DynamicFrame to a location in Amazon S3 and a table for it in the AWS Glue Data Catalog


In [7]:
# schema = br.get_schema()
s3_path = "s3://robin-poc-bucket/rahul/test_1/hands_json_data/data_1.json"
df = spark.readStream.format("json").schema(schema).load(s3_path)

In [8]:
df.printSchema()

root
 |-- event_data: struct (nullable = true)
 |    |-- community_cards: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- community_cards_rit: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- game_end_timestamp: integer (nullable = true)
 |    |-- game_start_timestamp: integer (nullable = true)
 |    |-- hand_id: long (nullable = true)
 |    |-- player_details: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cards: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |    |    |-- card_value: integer (nullable = true)
 |    

In [11]:
dfs1 = ( 
    df.select("*", "_metadata.file_modification_time", "_metadata.file_path").withColumn("record_date", expr("date(cast(event_timestamp as timestamp))"))
    )

In [12]:
dfs1.printSchema()

root
 |-- event_data: struct (nullable = true)
 |    |-- community_cards: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- community_cards_rit: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- game_end_timestamp: integer (nullable = true)
 |    |-- game_start_timestamp: integer (nullable = true)
 |    |-- hand_id: long (nullable = true)
 |    |-- player_details: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cards: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |    |    |-- card_value: integer (nullable = true)
 |    

In [ ]:
 # |-- _metadata: struct (nullable = true)
 # |    |-- file_path: string (nullable = true)
 # |    |-- file_name: string (nullable = true)
 # |    |-- file_size: long (nullable = true)
 # |    |-- file_modification_time: timestamp (nullable = true)

In [ ]:
sbronzeQuery = ( 
            df.writeStream
                    .queryName("bronze-1")
                    .format("delta")
                    .option("path", "s3://robin-poc-bucket/robin-poc-bucket/rahul/delta_database/bronze/")
                    .option("checkpointLocation", "s3://robin-poc-bucket/rahul/test_1/test_1_br/")
                    .outputMode("append")
                    .option("mergeSchema", "true")
                    .partitionBy("event_name", "record_date")
                    .toTable("pokershot_sample_delta_database.hands_bronze_data_test_4")
        ) 

In [5]:
spark.sql(f"Create database if not exists test_delta location 's3://robin-poc-bucket/rahul/delta_v2' ").show()

++
||
++
++


In [6]:
schema = SCHEMA = StructType([
            StructField("event_data", StructType([
                StructField("community_cards", ArrayType(StructType([
                    StructField("card_suit", StringType()),
                    StructField("card_value", IntegerType())
                ]))),
                StructField("community_cards_rit", ArrayType(StructType([
                    StructField("card_suit", StringType()),
                    StructField("card_value", IntegerType())
                ]))),
                StructField("game_end_timestamp", IntegerType()),
                StructField("game_start_timestamp", IntegerType()),
                StructField("hand_id", LongType()),
                StructField("player_details", ArrayType(StructType([
                    StructField("cards", ArrayType(StructType([
                        StructField("card_suit", StringType()),
                        StructField("card_value", IntegerType())
                    ]))),
                    StructField("chip_stack_end", DoubleType()),
                    StructField("chip_stack_start", DoubleType()),
                    StructField("position", StringType()),
                    StructField("profit_loss", DoubleType()),
                    StructField("reference_number", StringType()),
                    StructField("seat_index", IntegerType()),
                    StructField("session_id", StringType()),
                    StructField("total_bets", DoubleType()),
                    StructField("total_win", DoubleType()),
                    StructField("user_id", LongType()),
                    StructField("username", StringType()),
                    StructField("rake", StructType([
                        StructField("bonus_rake", DoubleType()),
                        StructField("net_rake", DoubleType()),
                        StructField("service_tax", DoubleType()),
                        StructField("total_rake", DoubleType())
                    ]))
                ]))),
                StructField("players_in_hand", IntegerType()),
                StructField("rit_game", IntegerType()),
                StructField("round_details", StructType([
                    StructField("flop", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ])),
                    StructField("preflop", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ])),
                    StructField("river", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ])),
                    StructField("showdown", StructType([
                        StructField("pot_details", ArrayType(DoubleType())),
                        StructField("pot_details_rit", ArrayType(DoubleType())),
                        StructField("pot_winners", ArrayType(StructType([
                            StructField("pot_amount", DoubleType()),
                            StructField("pot_contestants", ArrayType(IntegerType())),
                            StructField("winner_details", ArrayType(StructType([
                                StructField("amount", DoubleType()),
                                StructField("timestamp", IntegerType()),
                                StructField("user_id", IntegerType()),
                                StructField("win_type", StringType())
                            ])))
                        ]))),
                        StructField("pot_winners_rit", ArrayType(StructType([
                            StructField("pot_amount", DoubleType()),
                            StructField("pot_contestants", ArrayType(IntegerType())),
                            StructField("winner_details", ArrayType(StructType([
                                StructField("amount", DoubleType()),
                                StructField("timestamp", IntegerType()),
                                StructField("user_id", IntegerType()),
                                StructField("win_type", StringType())
                            ])))
                        ])))
                    ])),
                    StructField("turn", StructType([
                        StructField("player_action", ArrayType(StructType([
                            StructField("action", StringType()),
                            StructField("amount", DoubleType()),
                            StructField("chip_stack_end", DoubleType()),
                            StructField("chip_stack_start", DoubleType()),
                            StructField("position", StringType()),
                            StructField("timestamp", IntegerType()),
                            StructField("total_pot", DoubleType()),
                            StructField("user_id", IntegerType()),
                            StructField("username", StringType())
                        ]))),
                        StructField("pot_details", ArrayType(DoubleType()))
                    ]))
                ])),
                StructField("showdown", BooleanType()),
                StructField("table_info", StructType([
                    StructField("big_blind", DoubleType()),
                    StructField("is_anonymous", BooleanType()),
                    StructField("max_buyin", DoubleType()),
                    StructField("min_buyin", DoubleType()),
                    StructField("mini_game_type_id", StringType()),
                    StructField("players_per_table", IntegerType()),
                    StructField("small_blind", DoubleType()),
                    StructField("table_child_id", IntegerType()),
                    StructField("table_id", IntegerType()),
                    StructField("table_type", StringType())
                ])),
                StructField("total_pot", DoubleType()),
                StructField("user_stats", ArrayType(StructType([
                    StructField("3bet", BooleanType()),
                    StructField("3bet_opportunity", BooleanType()),
                    StructField("all_in", BooleanType()),
                    StructField("allin_opportunity", BooleanType()),
                    StructField("flop_seen", BooleanType()),
                    StructField("fold_opportunity", BooleanType()),
                    StructField("hp", BooleanType()),
                    StructField("pfr_count", BooleanType()),
                    StructField("pfr_opportunity", BooleanType()),
                    StructField("pr", BooleanType()),
                    StructField("time_played", IntegerType()),
                    StructField("user_id", IntegerType()),
                    StructField("user_won", BooleanType()),
                    StructField("vpip", BooleanType()),
                    StructField("vpip_opportunity", BooleanType()),
                    StructField("won_at_showdown", BooleanType()),
                    StructField("wtsd", BooleanType()),
                    StructField("fold_on_allin_opportunity", BooleanType()),
                    StructField("fold", BooleanType()),
                    StructField("3bet_fold_opportunity", BooleanType()),
                    StructField("steal_opportunity", BooleanType()),
                    StructField("steal_attempt", BooleanType()),
                    StructField("fold_on_allin", BooleanType()),
                    StructField("check_n_raise_opportunity", BooleanType()),
                    StructField("cbet_opportunity", BooleanType()),
                    StructField("cbet_success", BooleanType()),
                    StructField("fold_to_cbet_opportunity", BooleanType()),
                    StructField("folded_to_cbet", BooleanType()),
                    StructField("folded_to_3bet", BooleanType()),
                    StructField("check_n_raise", BooleanType())
                ]))),
                StructField("game_rake", StructType([
                    StructField("bonus_rake", DoubleType()),
                    StructField("net_rake", DoubleType()),
                    StructField("service_tax", DoubleType()),
                    StructField("total_rake", DoubleType())
                ]))
            ])),
            StructField("event_name", StringType()),
            StructField("event_source", StringType()),
            StructField("event_timestamp", IntegerType()),
            StructField("event_uuid", StringType()),
            StructField("event_version", StringType())
        ])

In [9]:
dfs = spark.readStream.format("json").schema(schema).load("s3://robin-poc-bucket/rahul/test_1/hands_json_data/")

In [10]:
dfs.printSchema()

root
 |-- event_data: struct (nullable = true)
 |    |-- community_cards: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- community_cards_rit: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- game_end_timestamp: integer (nullable = true)
 |    |-- game_start_timestamp: integer (nullable = true)
 |    |-- hand_id: long (nullable = true)
 |    |-- player_details: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cards: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |    |    |-- card_value: integer (nullable = true)
 |    

In [11]:
dfs_2 = dfs.select("*", "_metadata").withColumn("record_date", expr("date(cast(event_timestamp as timestamp))"))

In [12]:
dfs_2.printSchema()

root
 |-- event_data: struct (nullable = true)
 |    |-- community_cards: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- community_cards_rit: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- game_end_timestamp: integer (nullable = true)
 |    |-- game_start_timestamp: integer (nullable = true)
 |    |-- hand_id: long (nullable = true)
 |    |-- player_details: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cards: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |    |    |-- card_value: integer (nullable = true)
 |    

In [20]:
additional_options = {
    "path": "s3://robin-poc-bucket/rahul/delta_v2/hands_data_v2/"
}

# s3://robin-poc-bucket/rahul/delta_v2

In [21]:
squery = ( dfs_2.writeStream
                    .queryName("silver-processing-1")
                    .format("delta")
                    .options(**additional_options)
                    .option("checkpointLocation", "s3://robin-poc-bucket/rahul/test_1/test_2_br/")
                    .outputMode("append")
                    .option("mergeSchema", "true")
                    .partitionBy("event_name", "record_date")
                    .toTable("test_delta.hands_data_v2")
         )




In [18]:
%%sql
drop TABLE `pokershot_sample_delta_database_v1`.`hands_bronze_data` 

++
||
++
++


In [8]:
path = f"s3://robin-poc-bucket/rahul/pokershot_sample_delta_database_v1/test_2/"

In [10]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS pokershot_sample_delta_database_v1.test_2 (
        user_id bigint,
        record_date date,
        session_id string,
        minigames_type_id int,
        big_blind double,
        small_blind double,
        number_of_hands bigint,
        session_duration string,
        session_start_time int,
        session_end_time int,
        winning_hands bigint,
        losing_hands bigint,
        win_amount double
    )
    using delta
    location '{path}'
""")

DataFrame[]


In [20]:
df = spark.read.table("pokershot_sample_delta_database_v1.hands_bronze_data")

In [21]:
df.show(2)

+--------------------+---------------+------------+---------------+--------------------+-------------+----------------------+--------------------+-----------+
|          event_data|     event_name|event_source|event_timestamp|          event_uuid|event_version|file_modification_time|           file_path|record_date|
+--------------------+---------------+------------+---------------+--------------------+-------------+----------------------+--------------------+-----------+
|{[{DIAMOND, 8}, {...|GAME_STATS_CASH|         NEW|     1704476070|51b59aee-37a1-4b6...|       v5.1.6|   2024-01-15 10:31:53|s3://robin-poc-bu...| 2024-01-05|
|{[{HEART, 3}, {HE...|GAME_STATS_CASH|         NEW|     1704476075|d8dcad19-f59e-4f8...|       v5.1.6|   2024-01-15 10:31:53|s3://robin-poc-bu...| 2024-01-05|
+--------------------+---------------+------------+---------------+--------------------+-------------+----------------------+--------------------+-----------+
only showing top 2 rows


In [23]:
hands_data_df = ( 
            spark.readStream
                .option("startingTimestamp", "2024-01-01")
                .table("pokershot_sample_delta_database_v1.hands_bronze_data")  
            )

In [25]:
hands_data_df = hands_data_df.withColumn("player_details", expr("event_data.player_details"))
flattern_hands_data_df  = hands_data_df.selectExpr(
    "event_data.hand_id as hand_id", 
    "event_data.game_end_timestamp as game_end_timestamp",
    "event_data.game_start_timestamp as game_start_timestamp",
    "event_data.table_info.big_blind as big_blind",
    "case when event_data.table_info.big_blind = 0 then 0 when event_data.table_info.big_blind = 5 then 2 when event_data.table_info.big_blind = 25 then 10 else event_data.table_info.big_blind/2 end as small_blind",
    "event_name", 
    "event_source", 
    "cast(event_timestamp as timestamp) as event_timestamp", 
    "event_uuid", 
    "event_version",
    "explode(player_details) as player_detail", 
    "date(cast(event_timestamp as timestamp)) as record_date",
    "player_detail.user_id as user_id",
    "player_detail.profit_loss as profit_loss",
    "player_detail.position as position",
    "player_detail.total_bets as stake",
    "player_detail.rake.total_rake as total_rake",
    "player_detail.chip_stack_start as chip_stack_start",
    "player_detail.chip_stack_end as chip_stack_end",
    "player_detail.reference_number as session_id",
    "case when player_detail.total_bets + player_detail.profit_loss = 0 then 0 else  player_detail.total_bets + player_detail.profit_loss + player_detail.rake.total_rake end as win",
    """case when (replace(cast(event_data.table_info.mini_game_type_id as string), '"',"")) in ("Holdem") then 1  when (replace(cast(event_data.table_info.mini_game_type_id as string), '"',"")) in ('Omaha') then 2  when (replace(cast(event_data.table_info.mini_game_type_id as string), '"',"")) in ("FiveCardPLO") then 17  when (replace(cast(event_data.table_info.mini_game_type_id as string), '"',"")) in ('SixCardPLO') then 20  when (replace(cast(event_data.table_info.mini_game_type_id as string), '"',"")) in ("SuperHoldem") then 21  else 25 end as minigames_type_id"""
).drop("player_detail")

In [26]:
df_performance_by_stake = flattern_hands_data_df.withWatermark("event_timestamp", "1440 minutes") \
    .groupBy(
        "user_id", "record_date", "session_id", "minigames_type_id", "big_blind", "small_blind"
    ).agg(
        count("*").alias("number_of_hands"),
        expr("cast((max(game_end_timestamp) - min(game_start_timestamp)) as string)").alias("session_duration"),
        min("game_start_timestamp").alias("session_start_time"),
        max("game_start_timestamp").alias("session_end_time"),
        sum(when((col("win") - col("stake")) > 0, 1).otherwise(0)).alias("winning_hands"),
        sum(when((col("win") - col("stake")) < 0, 1).otherwise(0)).alias("losing_hands"),
        sum("profit_loss").alias("win_amount")
    )

In [27]:
def upsert(gold_aggregate_df, batch_id):
    
    gold_aggregate_df.createOrReplaceTempView("tmp_hands_agg")
    merge_statement = f"""
        MERGE INTO pokershot_sample_delta_database_v1.test_2 t
        USING tmp_hands_agg s
            ON  ( 
                s.user_id == t.user_id and 
                s.record_date = t.record_date and 
                s.session_id = t.session_id and 
                s.minigames_type_id = t.minigames_type_id and 
                s.big_blind = t.big_blind and 
                s.small_blind = t.small_blind
            )
        WHEN MATCHED THEN
            UPDATE SET *
        WHEN NOT MATCHED THEN
            INSERT *
        """
    gold_aggregate_df._jdf.sparkSession().sql(merge_statement) 

In [30]:
def saveToTable(gold_aggregate_df):
    sGoldUpdateQuery = (   gold_aggregate_df.writeStream
            .queryName("gold-update")
            .option("checkpointLocation", "s3://robin-poc-bucket/rahul/test_1/md/")
            .outputMode("update")
            .foreachBatch(upsert)
            .start()
    )
    return sGoldUpdateQuery

In [31]:
x = saveToTable(df_performance_by_stake)

In [24]:
hands_data_df.printSchema()

root
 |-- event_data: struct (nullable = true)
 |    |-- community_cards: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- community_cards_rit: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |-- card_value: integer (nullable = true)
 |    |-- game_end_timestamp: integer (nullable = true)
 |    |-- game_start_timestamp: integer (nullable = true)
 |    |-- hand_id: long (nullable = true)
 |    |-- player_details: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- cards: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- card_suit: string (nullable = true)
 |    |    |    |    |    |-- card_value: integer (nullable = true)
 |    

In [15]:
%%sql
drop table `pokershot_sample_delta_database_v1`.`test`

++
||
++
++


In [18]:
%%sql
SELECT * FROM `pokershot_sample_delta_database_v1`.`hands_bronze_data`;

+--------------------+---------------+------------+---------------+--------------------+-------------+----------------------+--------------------+-----------+
|          event_data|     event_name|event_source|event_timestamp|          event_uuid|event_version|file_modification_time|           file_path|record_date|
+--------------------+---------------+------------+---------------+--------------------+-------------+----------------------+--------------------+-----------+
|{[{DIAMOND, 8}, {...|GAME_STATS_CASH|         NEW|     1704476070|51b59aee-37a1-4b6...|       v5.1.6|   2024-01-15 10:31:53|s3://robin-poc-bu...| 2024-01-05|
|{[{HEART, 3}, {HE...|GAME_STATS_CASH|         NEW|     1704476075|d8dcad19-f59e-4f8...|       v5.1.6|   2024-01-15 10:31:53|s3://robin-poc-bu...| 2024-01-05|
|{[{SPADE, 11}, {H...|GAME_STATS_CASH|         NEW|     1704476086|cab36423-438a-481...|       v5.1.6|   2024-01-15 10:31:53|s3://robin-poc-bu...| 2024-01-05|
|{[{DIAMOND, 7}, {...|GAME_STATS_CASH|        